Runs import statements

In [1]:
import os
import re
import string
import datetime
import numpy as np
import pandas as pd

Finds files in the data directory

In [2]:
data_directory = os.getcwd() + '/april-october'

cwd, files, filenames = next(os.walk(data_directory))
print(cwd)
print(files)
print(filenames)

/Users/eddychiao/Documents/Georgia Tech/CS/GaTech-CS/MSCS/CS 6474/project/april-october
['title', 'comment']
['cleaned_titles.zip', '.DS_Store']


# Cleaning Titles

Reads data in specific csv (we'll just look at title.csv and comment.csv for now

In [10]:
title_cwd = cwd + '/title'
_, _, title_names = next(os.walk(title_cwd))
print(title_names)

['title_nov_2020.csv', 'title_Jan_2021.csv', '.DS_Store', 'title_Dec_2020.csv', 'title_Aug_2020.csv', 'title_Oct_2020.csv', 'title_Jun_2020.csv', 'title_April_2020.csv', 'title_Feb_2021.csv', 'title_Mar_2021.csv', 'title_Jul_2020.csv', 'title_Sep_2020.csv', 'title_May_2020.csv']


In [11]:
title_names = [file for file in title_names if file[-4:] == '.csv']
print(title_names)

# all_csvs = []title_names
# # reads all the csvs in the data folder
# for file in filenames:
#     all_csvs.append(file)

['title_nov_2020.csv', 'title_Jan_2021.csv', 'title_Dec_2020.csv', 'title_Aug_2020.csv', 'title_Oct_2020.csv', 'title_Jun_2020.csv', 'title_April_2020.csv', 'title_Feb_2021.csv', 'title_Mar_2021.csv', 'title_Jul_2020.csv', 'title_Sep_2020.csv', 'title_May_2020.csv']


In [12]:
# Shows preview of each csv
for csv in title_names:
    csv_title = pd.read_csv(title_cwd + '/' + csv)
    csv_title = csv_title.dropna()
    print(csv_title.head(5))
    print(csv_title.shape)
    print()

                                               Title        timestamp
0                            J POW'S PRINTER AWAITS!  2020-10-31 4:02
1                                            Yall rn  2020-10-31 4:02
2                                          TSLA PUTS  2020-10-31 4:03
3  TSLA Stonks Dipped Just To Scare You For Hallo...  2020-10-31 4:04
4                   My autistic ass after this week.  2020-10-31 4:11
(24644, 2)

                                               Title        timestamp
0       How my last day of trading in 2020 felt like  2021-01-01 3:59
1                          Me investing in anything.  2021-01-01 3:56
2                                The next yolo play?  2021-01-01 3:53
3  A visual on how my last day trading in 2020 fe...  2021-01-01 3:48
4  Got this succulent Chinese meal with this fort...  2021-01-01 3:47
(54279, 2)

                                               Title        timestamp
0               🚀🚀🚀🖕🏳️‍🌈🐻 pltr 12/4 40c avg cost .34  2020-12-01 3

In [13]:
for csv in title_names:
    print(csv)
    csv_title = pd.read_csv(title_cwd + '/' + csv)
    csv_title = csv_title.dropna()
    csv_title.Title = csv_title.Title.str.lower()
    
    title_data_cleaned = csv_title.copy()
    
    title_data_cleaned.Title = title_data_cleaned.Title.apply(lambda x:re.sub('@[^\s]+','',x))
    # Remove URLS
    title_data_cleaned.Title = title_data_cleaned.Title.apply(lambda x:re.sub(r"http\S+", "", x))
    # Remove all the special characters
    title_data_cleaned.Title = title_data_cleaned.Title.apply(lambda x:' '.join(re.findall(r'\w+', x)))
    # Remove numbers
    title_data_cleaned.Title = title_data_cleaned.Title.apply(lambda x:re.sub(' \d+', '', x))
    # Remove all single characters
    title_data_cleaned.Title = title_data_cleaned.Title.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', ' ', x))
    # Remove single characters again because still leaving some
    title_data_cleaned.Title = title_data_cleaned.Title.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', ' ', x))
    # Removes specific time of day from timestamp
#     title_data_cleaned.timestamp = pd.to_datetime(title_data_cleaned.timestamp).dt.date
    # CUTOFFS: 0:00, 9:00, 13:00, 17:00

    title_data_cleaned.timestamp = pd.to_datetime(title_data_cleaned.timestamp).dt.floor('h')
#     print(title_data_cleaned.timestamp)


    title_data_cleaned = title_data_cleaned.loc[pd.to_datetime(title_data_cleaned.timestamp).dt.hour < 16]
    title_data_cleaned.loc[pd.to_datetime(title_data_cleaned.timestamp).dt.time < datetime.time(9, 30, 0), 'time'] = datetime.time(0)
    title_data_cleaned.loc[pd.to_datetime(title_data_cleaned.timestamp).dt.time >= datetime.time(9, 30, 0), 'time'] = datetime.time(9, 30, 0)
    title_data_cleaned.loc[pd.to_datetime(title_data_cleaned.timestamp).dt.time >= datetime.time(12, 30, 0), 'time'] = datetime.time(12, 30, 0)

    
#     title_data_cleaned.loc[pd.to_datetime(title_data_cleaned.timestamp).dt.hour >= 16, 'time'] = datetime.time(17)
#     print(type(title_data_cleaned.timestamp))

    title_data_cleaned.timestamp = pd.to_datetime(title_data_cleaned.timestamp).dt.date
    
    title_data_cleaned.timestamp = pd.to_datetime(title_data_cleaned.timestamp).dt.strftime('%Y-%m-%d')
    title_data_cleaned.time = title_data_cleaned.time.astype(str)
    
    
    title_data_cleaned.timestamp = pd.to_datetime(title_data_cleaned.timestamp + ' ' + title_data_cleaned.time)
    title_data_cleaned = title_data_cleaned.drop('time', axis=1)
    
    print(title_data_cleaned.head(5))
    print()
    
    # Export data to csv
    title_data_cleaned.to_csv(title_cwd + '/' + csv[:-4] + '_cleaned.csv')

title_nov_2020.csv
                                               Title  timestamp
0                               j pow printer awaits 2020-10-31
1                                            yall rn 2020-10-31
2                                          tsla puts 2020-10-31
3  tsla stonks dipped just to scare you for hallo... 2020-10-31
4                    my autistic ass after this week 2020-10-31

title_Jan_2021.csv
                                               Title  timestamp
0            how my last day of trading in felt like 2021-01-01
1                           me investing in anything 2021-01-01
2                                 the next yolo play 2021-01-01
3   a visual on how my last day trading in felt like 2021-01-01
4  got this succulent chinese meal with this fort... 2021-01-01

title_Dec_2020.csv
                                               Title  timestamp
0                                     pltrc avg cost 2020-12-01
1  my tesla calls at close gain loss porn in 

In [14]:
print(title_data_cleaned[:19])

                                                Title  timestamp
0                                1st thing bring lube 2020-05-02
1         the bond market crash begins on mayth andth 2020-05-02
2                    make insider trading legal again 2020-05-02
3                                             retreat 2020-05-02
4                                      u bryguysayshi 2020-05-02
5   i should of held ended up selling all for brea... 2020-05-02
6   this documentary about autism made me think of... 2020-05-02
7   why is there such price difference on internat... 2020-05-02
8   just thought this should be reposted based of ... 2020-05-02
9                               better to spxu or put 2020-05-02
10          maybe elon will save me whatever it takes 2020-05-02
11                                     this aged well 2020-05-02
12                            attention all wsb users 2020-05-02
13                        the market isnt the economy 2020-05-02
14           question abo

# Clean Comments

In [ ]:
comment_cwd = cwd + '/comment'
_, _, comment_names = next(os.walk(comment_cwd))
print(comment_names)

In [ ]:
comment_names = [file for file in comment_names if file[-4:] == '.csv']
print(comment_names)

In [ ]:
# Shows preview of each csv
for csv in comment_names:
    csv_comment = pd.read_csv(comment_cwd + '/' + csv)
    csv_comment = csv_comment.dropna()
    print(csv_comment.head(5))
    print(csv_comment.shape)
    print()

In [ ]:
# Shows preview of each csv
for csv in comment_names:
    csv_comment = pd.read_csv(comment_cwd + '/' + csv)
    csv_comment = csv_comment.dropna()
    csv_comment.Comment = csv_comment.Comment.str.lower()
    
    comment_data_cleaned = csv_comment.copy()
    
#     print(comment_data_cleaned)
    
    comment_data_cleaned.Comment = comment_data_cleaned.Comment.apply(lambda x:re.sub('@[^\s]+','',x))
    # Remove URLS
    comment_data_cleaned.Comment = comment_data_cleaned.Comment.apply(lambda x:re.sub(r"http\S+", "", x))
    # Remove all the special characters
    comment_data_cleaned.Comment = comment_data_cleaned.Comment.apply(lambda x:' '.join(re.findall(r'\w+', x)))
    # Remove numbers
    comment_data_cleaned.Comment = comment_data_cleaned.Comment.apply(lambda x:re.sub(' \d+', '', x))
    # Remove all single characters
    comment_data_cleaned.Comment = comment_data_cleaned.Comment.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', ' ', x))
    # Remove single characters again because still leaving some
    comment_data_cleaned.Comment = comment_data_cleaned.Comment.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', ' ', x))
    # Removes specific time of day from timestamp
#     comment_data_cleaned.timestamp = pd.to_datetime(csv_comment.timestamp).dt.date
    
    comment_data_cleaned = comment_data_cleaned[comment_data_cleaned.Comment != 'deleted']
    comment_data_cleaned = comment_data_cleaned[comment_data_cleaned.Comment != 'removed']
    csv_comment = csv_comment.dropna()
    
    # CUTOFFS: 0:00, 9:00, 13:00, 17:00

    comment_data_cleaned.timestamp = pd.to_datetime(comment_data_cleaned.timestamp).dt.floor('H')
#     print(title_data_cleaned.timestamp)
    comment_data_cleaned.loc[pd.to_datetime(comment_data_cleaned.timestamp).dt.hour < 9, 'time'] = datetime.time(0)
    comment_data_cleaned.loc[pd.to_datetime(comment_data_cleaned.timestamp).dt.hour >= 9, 'time'] = datetime.time(9)
    comment_data_cleaned.loc[pd.to_datetime(comment_data_cleaned.timestamp).dt.hour >= 13, 'time'] = datetime.time(13)
    comment_data_cleaned.loc[pd.to_datetime(comment_data_cleaned.timestamp).dt.hour >= 17, 'time'] = datetime.time(17)
#     print(type(comment_data_cleaned.timestamp)comment_data_cleaned)

    comment_data_cleaned.timestamp = pd.to_datetime(comment_data_cleaned.timestamp).dt.date
    
    comment_data_cleaned.timestamp = pd.to_datetime(comment_data_cleaned.timestamp).dt.strftime('%Y-%m-%d')
    comment_data_cleaned.time = comment_data_cleaned.time.astype(str)
    
    
    comment_data_cleaned.timestamp = pd.to_datetime(comment_data_cleaned.timestamp + ' ' + comment_data_cleaned.time)
#     comment_data_cleaned = comment_data_cleaned.drop('time', axis=1)
    
    
    
    # Export data to csv
    comment_data_cleaned.to_csv(comment_cwd + '/' + csv[:-4] + '_cleaned.csv')
    print(comment_data_cleaned.head(5))
    print()